<a href="https://colab.research.google.com/github/Ieejuyeon/AlgorithmStudy/blob/DataPreprocessing/%EB%B0%9C%EC%9D%8C%EC%98%A4%EB%A5%98%EB%B3%80%ED%99%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install --upgrade transformers
import transformers
print(transformers.__version__)

4.51.3


In [1]:
import zipfile
import os

# Google Drive 내 ZIP 파일 경로
zip_file = 'drive/MyDrive/2025인공지능6팀/LcmsExcelView (1).zip'

# 압축을 풀 디렉토리 설정
extract_dir = 'drive/MyDrive/2025인공지능6팀/Lcms'

# 디렉토리가 없으면 생성
os.makedirs(extract_dir, exist_ok=True)

# ZIP 파일 해제
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# 압축 해제된 파일 목록 확인
extracted_files = os.listdir(extract_dir)
print("압축 해제된 파일들:", extracted_files)


KeyboardInterrupt: 

In [9]:

import os
file_name = "drive/MyDrive/2025인공지능6팀/LcmsExcelView (1).zip"
output_dir = "drive/MyDrive/2025인공지능6팀/Lcms"
os.system("unzip "+file_name+" -d "+output_dir)

512

데이터 준비

In [2]:
import transformers
from transformers import Seq2SeqTrainingArguments

from transformers import (
    PreTrainedTokenizerFast,
    BartForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)
import torch

# 1. 모델과 토크나이저 로딩
model_name = "hyunwoongko/kobart"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/337 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/109 [00:00<?, ?B/s]

KeyboardInterrupt: 

In [17]:
import pandas as pd
from datasets import Dataset

# 2. 데이터셋 로딩
dfs = []

for i in range(1, 31):
    file_name = f"drive/MyDrive/2025인공지능6팀/Lcms/LcmsExcelView ({i}).xlsx"
    try:
        tempdf = pd.read_excel(file_name, engine='openpyxl', header=2, usecols=['원 형태', '교정 형태','앞 문맥', '중심어', '뒤 문맥'])
        dfs.append(tempdf)
    except Exception as e:
        print(f"[{i}] 파일 오류: {file_name} → {type(e).__name__}: {e}")

df = pd.concat(dfs, ignore_index=True)
df["원 형태"] = df["원 형태"].astype(str)
df["교정 형태"] = df["교정 형태"].astype(str)
dataset = Dataset.from_pandas(df.reset_index(drop=True))

df.loc[df['앞 문맥']=='{좌 문맥 없음}', '앞 문맥'] = ''
df.loc[df['뒤 문맥']=='{우 문맥 없음}', '뒤 문맥'] = ''

# 그룹별로 원 형태와 교정 형태를 문자열로 연결 (순서는 기존 행 순서 유지)
grouped = df.groupby(['앞 문맥', '뒤 문맥']).agg({
    '원 형태': lambda x: ''.join(x),         # 연결할 때 구분자 넣고 싶으면 ''.join → ' '.join(x) 등으로 변경
    '교정 형태': lambda x: ''.join(x)
}).reset_index()
# 그룹 사이즈 계산
counts = df.groupby(['앞 문맥', '뒤 문맥']).size().reset_index(name='counts')

# 2개 이상인 그룹만 필터링
valid_groups = counts[counts['counts'] >= 2]

# valid_groups와 grouped를 병합해서 조건 맞는 그룹 결과만 추출
result = grouped.merge(valid_groups[['앞 문맥', '뒤 문맥']], on=['앞 문맥', '뒤 문맥'])

print(result)


/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default sty

[5] 파일 오류: drive/MyDrive/2025인공지능6팀/Lcms/LcmsExcelView (5).xlsx → BadZipFile: File is not a zip file


/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default sty

                                       앞 문맥                  뒤 문맥  \
0                                                                   
1                                                           강화합니다   
2                                            관리를 잘 지킨 지키자고 생각합니다.   
3                                                       교육은 무엇집니까   
4                                                         근데 어떤 거   
..                                      ...                   ...   
151  친 제 친한 찐꾸가 같이 하자고 해서 아 아 제 뽕사 환동은 지원했고                 받았습니다   
152                                 하 할 거 할                         
153                               한국어 더 잘하고                         
154                                      해서        대한 경찰의 발표에 타르면   
155                                      흙과            다 옮겨야 했습니다   

                                                  원 형태  \
0    믄그게가지매니지먼트복고레인안국이닥이타회사쓴차람감싸꺼피숍엠스물르씨칭구학쿄엠캄사항쌍어가...   
1                                                  꾸쭉끄 

In [ ]:

# 3. 전처리 함수 정의
max_length = 64

def preprocess(example):
    input_enc = tokenizer(text=example["원 형태"], truncation=True, padding="max_length", max_length=max_length)
    target_enc = tokenizer(text_target=example["교정 형태"], truncation=True, padding="max_length", max_length=max_length)

    input_enc["labels"] = target_enc["input_ids"]
    return input_enc

tokenized_dataset = dataset.map(preprocess)

# 4. 데이터 분할
split_dataset = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

# 5. 트레이닝 설정
training_args = Seq2SeqTrainingArguments(
    output_dir="./kobart-corrector",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),  # GPU 사용 시 FP16
)

# 6. 트레이너 준비
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# 7. 학습 시작
trainer.train()
trainer.save_model("./kobart-corrector")  # 수동 저장
tokenizer.save_pretrained("./kobart-corrector")


In [ ]:
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

# 1. 모델과 토크나이저 로드 (경로는 학습된 모델 위치 또는 허깅페이스 경로)
model = BartForConditionalGeneration.from_pretrained("./kobart-corrector")
tokenizer = PreTrainedTokenizerFast.from_pretrained("./kobart-corrector")

# 2. 테스트 입력 문장
input_text = "목건이"

# 3. 토크나이즈 및 인코딩
inputs = tokenizer(input_text, return_tensors="pt")

# 4. 모델 예측 (디코딩)
output_ids = model.generate(**inputs, max_length=32)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("입력:", input_text)
print("출력:", output_text)
